# Importing Data from CSV

This notebooks reads sample data from `data/emp.csv` and we are trying to run some basic pyspark commands and work on the data!

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 5.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 13.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=2db838069e726635876d9c19417397a134267881ce26071d1fe93c77a4cb9c0a
  Stored in directory: /home/codespace/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CSV data").getOrCreate()

23/06/14 07:21:57 WARN Utils: Your hostname, codespaces-b6d6b5 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/06/14 07:21:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/14 07:21:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
df_emp_csv  = spark.read.option("nullValue","null").csv("data/emp.csv",header=True,inferSchema=True)

In [8]:


df = spark.read.csv("/workspaces/codespaces-jupyter/data/emp.csv", header=True, inferSchema=True)
display(df)

DataFrame[EMPNO: string, ENAME: string, JOB: string, MGR: string, HIREDATE: string, SAL: string, COMM: string, DEPTNO: string, UPDATED_DATE: string]

In [9]:
from pyspark.sql.functions import to_date
#Change string to Date DataType
df = df.withColumn("HIREDATE",to_date("HIREDATE",'dd-MM-yyyy')).fillna({"HIREDATE":'9999-12-31'})
df.show()

+-----+------+---------+----+----------+----+----+------+------------+
|EMPNO| ENAME|      JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|
+-----+------+---------+----+----------+----+----+------+------------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|  2022-01-01|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|  2022-01-01|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|  2022-01-01|
| 7566| JONES|  MANAGER|7839|1981-02-04|2975|null|    20|  2022-01-05|
| 7654|MARTIN| SALESMAN|7698|1981-09-21|1250|1400|    30|  2022-01-03|
| 7698|   SGR|  MANAGER|7839|1981-01-05|2850|null|    30|  2022-01-04|
| 7782|  RAVI|  MANAGER|7839|1981-09-06|2450|null|    10|  2022-01-02|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|  2022-01-02|
| 7839|  KING|PRESIDENT|null|1981-11-01|5000|null|    10|  2022-01-02|
| 7844|TURNER| SALESMAN|7698|1981-08-09|1500|   0|    30|  2022-01-02|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|null|    20|  2022-01-03|
| 7900

In [10]:

from pyspark.sql.functions import date_format
#creating two YEAR and MONTH new columns based on hiredate date field
df_emp_csv = df.withColumn("YEAR",date_format("HIREDATE",'yyyy')).withColumn("MONTH",date_format("HIREDATE",'MM'))
df_emp_csv.show()

+-----+------+---------+----+----------+----+----+------+------------+----+-----+
|EMPNO| ENAME|      JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|YEAR|MONTH|
+-----+------+---------+----+----------+----+----+------+------------+----+-----+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|  2022-01-01|1980|   12|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|  2022-01-01|1981|   02|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|  2022-01-01|1981|   02|
| 7566| JONES|  MANAGER|7839|1981-02-04|2975|null|    20|  2022-01-05|1981|   02|
| 7654|MARTIN| SALESMAN|7698|1981-09-21|1250|1400|    30|  2022-01-03|1981|   09|
| 7698|   SGR|  MANAGER|7839|1981-01-05|2850|null|    30|  2022-01-04|1981|   01|
| 7782|  RAVI|  MANAGER|7839|1981-09-06|2450|null|    10|  2022-01-02|1981|   09|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|  2022-01-02|1987|   04|
| 7839|  KING|PRESIDENT|null|1981-11-01|5000|null|    10|  2022-01-02|1981|   11|
| 7844|TURNER| S

In [11]:

# using PartitionBy with two columns creating partitions
df_emp_csv.write.format("delta").partitionBy("YEAR","MONTH").mode("overwrite").saveAsTable("emp_part")

Py4JJavaError: An error occurred while calling o102.saveAsTable.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:738)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:860)
	at org.apache.spark.sql.DataFrameWriter.saveAsTable(DataFrameWriter.scala:559)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAc